In [ ]:
# O objetivo final é captar apenas os contatos não salvos, e criar um arquivo csv. Posteriormente, importa-los para a agenda do google, e utiliza-los
# em listas para marketing no whatsapp.
# Esse código de modo geral, vai retornar uma lista com todos os contatos da sua lista de chats do whatsapp (salvos ou não. Vai printando conforme
# for captando).
# Tem uma função embutida para verificar se o contato é um numero ou nome, e armazena-lo em uma lists

# ATENÇÃO, ANTES DE INICIAR O SCRIPT CERTIFIQUE-SE DE QUE TODO O AMBIENTE, JUNTAMENTE COM O DISPOSITIVOS ESTEJAM CONFIGURADOS CORREAMENTE
# TODOS OS CÓDIGOS FORAM GERADOS POR INTELIGÊNCIA ARTIFICIAL, COM DIRECIONAMENTO E ORIENTAÇÃO HUMANA

import re
from appium import webdriver
from appium.options.android import UiAutomator2Options
from time import sleep
from selenium.common.exceptions import NoSuchElementException

# Configuração das opções para o Appium
options = UiAutomator2Options()
options.platform_name = "Android"
options.device_name = "PT99621EA1A32000599"  # Substitua pelo ID do dispositivo listado no 'adb devices'
options.app_package = "com.whatsapp.w4b"  # Nome do pacote do WhatsApp Business
options.app_activity = "com.whatsapp.HomeActivity"  # Atividade principal
options.no_reset = True  # Para evitar que o app seja reiniciado

# Conectar ao Appium Server
driver = webdriver.Remote("http://localhost:4727/wd/hub", options=options)
sleep(5)  # Aguarda o aplicativo carregar

# Função para rolar a tela com coordenadas dinâmicas
def scroll_down():
    screen_size = driver.get_window_size()
    start_x = screen_size['width'] // 2
    start_y = screen_size['height'] * 4 // 5  # 80% da altura da tela
    end_y = screen_size['height'] * 1 // 5  # 20% da altura da tela
    driver.swipe(start_x, start_y, start_x, end_y, 1000)  # Duração ajustada para 1 segundo
    sleep(1)

# Função para verificar se o texto é um número de telefone
def is_phone_number(text):
    return re.match(r'^\+55\s?\d{2}\s?\d{4,5}[-\s]?\d{4}$', text)

# Lista para armazenar apenas os números de telefone encontrados
phone_numbers = set()
all_contacts = set()  # Lista para armazenar todos os contatos visíveis

try:
    while True:
        # Verifica se o app está ativo
        if driver.current_package != "com.whatsapp.w4b":
            print("O aplicativo foi fechado. Tentando reabrir...")
            driver.start_activity("com.whatsapp.w4b", "com.whatsapp.HomeActivity")
            sleep(5)

        # Localiza todas as conversas visíveis na tela
        conversations = driver.find_elements(
            "xpath",
            '//android.widget.TextView[@resource-id="com.whatsapp.w4b:id/conversations_row_contact_name"]'
        )

        # Extrai os contatos visíveis na tela
        new_contacts_found = False
        for conversation in conversations:
            contact_name = conversation.get_attribute("text")
            if contact_name not in all_contacts:  # Evita duplicatas
                all_contacts.add(contact_name)
                new_contacts_found = True
                print(f"Contato encontrado: {contact_name}")

                # Adiciona apenas números válidos na lista de números de telefone
                if is_phone_number(contact_name):
                    phone_numbers.add(contact_name)

        # Verifica se a rolagem trouxe novos contatos
        if not new_contacts_found:
            print("Nenhum novo contato encontrado. Fim da lista.")
            break

        # Rolar a tela para capturar mais contatos
        scroll_down()

except NoSuchElementException:
    print("Fim da lista de conversas ou nenhum elemento encontrado.")

# Mostra o total de contatos capturados
print(f"Total de contatos capturados: {len(all_contacts)}")
print(f"Total de números válidos capturados: {len(phone_numbers)}")

# Encerrar a sessão do Appium
driver.quit()


In [ ]:
# Esse código cria um arquivo .csv a partir da lista de numeros de telefone phone_list().
# Tambem adiciona nomes junto a cada número, (Genéricos como "Cliente e um número", mas fica a critério). 

import csv

# Nome do arquivo CSV intermediário
intermediate_csv_file = "contatos_intermediarios.csv"

# Lista de números capturados (substitua pela variável 'phone_numbers' gerada anteriormente)
# Exemplo: phone_numbers = ["+55 64 9949-7910", "+55 62 8406-7007"]
phone_numbers = list(phone_numbers)  # Certifique-se de que esta variável está carregada

# Criar arquivo CSV intermediário
with open(intermediate_csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    
    # Cabeçalhos básicos para identificação
    writer.writerow(["Name", "Phone"])

    # Preencher o CSV com os contatos
    cliente_number = 122
    for number in phone_numbers:
        writer.writerow([f"Cliente {cliente_number}", number])
        cliente_number += 1

print(f"Arquivo CSV intermediário criado com sucesso: {intermediate_csv_file}")


In [ ]:
# Esse cria um outro arquivo .csv com os contatos formatados de acordo com os requisitos da agenda do google

import csv

# Nome do arquivo CSV intermediário (entrada)
intermediate_csv_file = "contatos_intermediarios.csv"

# Nome do arquivo CSV de saída (formato Google Contatos)
output_csv_file = "contatos_google_formatado.csv"

# Lista para armazenar os contatos com nomes e números
clientes = []

# Ler os dados do arquivo intermediário
with open(intermediate_csv_file, mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # Pula o cabeçalho
    for row in reader:
        if row:  # Ignorar linhas vazias
            clientes.append({"name": row[0], "phone": row[1]})

# Criar o arquivo CSV no formato esperado pelo Google Contatos
with open(output_csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    # Cabeçalhos completos exportados do Google Contatos
    writer.writerow([
        "First Name", "Middle Name", "Last Name", "Phonetic First Name", "Phonetic Middle Name",
        "Phonetic Last Name", "Name Prefix", "Name Suffix", "Nickname", "File As",
        "Organization Name", "Organization Title", "Organization Department", "Birthday", "Notes",
        "Photo", "Labels", "E-mail 1 - Label", "E-mail 1 - Value", "Phone 1 - Label", "Phone 1 - Value",
        "Phone 2 - Label", "Phone 2 - Value", "Phone 3 - Label", "Phone 3 - Value",
        "Address 1 - Label", "Address 1 - Formatted", "Address 1 - Street", "Address 1 - City",
        "Address 1 - PO Box", "Address 1 - Region", "Address 1 - Postal Code", "Address 1 - Country",
        "Address 1 - Extended Address", "Website 1 - Label", "Website 1 - Value"
    ])

    # Adicionar os contatos formatados, preenchendo apenas os campos essenciais
    for cliente in clientes:
        writer.writerow([
            cliente["name"], "", "", "", "", "", "", "", "", "",
            "", "", "", "", "", "", "", "", "", "Mobile", cliente["phone"],
            "", "", "", "", "", "", "", "", "", "", "", ""
        ])

print(f"Arquivo CSV formatado criado com sucesso: {output_csv_file}")


In [ ]:
# Após finalizar os passos desse script, basta importar o arquivo csv gerado no último código para os contatos do google pelo navegador mesmo
# (provavelmente está no mesmo diretorio do arquivo de script) e esperar atualizar a agenda do whatsapp.
# Depois disso basta disparar as mensagens para seus contatos conforme preferir.